In [1]:
contest_folder = 'practice'

In [2]:
from analysis import get_current_status, process_row

In [3]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [4]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [5]:
aggregated_df

""


In [7]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=process_row, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Not outputs found yet
Running analysis at 3108
Running analysis at 3108 for Line by Line
Running analysis at 3138
Running analysis at 3138 for Fall in Line
Running analysis at 3138 for Line of Delivery (Part 2)
Running analysis at 3208
Running analysis at 3208 for Fall in Line
Running analysis at 3208 for Line of Delivery (Part 1)
Running analysis at 3208 for Line of Delivery (Part 2)
Running analysis at 3238
Running analysis at 3238 for Walk the Line
Running analysis at 3308
Running analysis at 3338
Running analysis at 3338 for Line of Delivery (Part 1)
Running analysis at 3408
Running analysis at 3408 for Walk the Line


KeyboardInterrupt: 